In [118]:
from __future__ import print_function
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import numpy as np
#import matplotlib
matplotlib.use('GtkAgg')
import matplotlib.pyplot as plt
import pandas as pd

/depot/chubykin/apps/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:8: UserWarning: 
This call to matplotlib.use() has no effect because the backend has already
been chosen; matplotlib.use() must be called *before* pylab, matplotlib.pyplot,
or matplotlib.backends is imported for the first time.

The backend was *originally* set to 'agg' by the following code:
  File "/depot/chubykin/apps/anaconda3/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/depot/chubykin/apps/anaconda3/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/depot/chubykin/apps/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/depot/chubykin/apps/anaconda3/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/depot/chubykin/apps/anaconda3/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 478

In [63]:
z = torch.randn(3,2)
print(z)
for i, input_t in enumerate(z.chunk(z.size(0), dim=1)):
    print(i)
    print(input_t)


-0.4516 -0.9455
-0.2610 -0.4676
-0.6422 -0.6638
[torch.FloatTensor of size 3x2]

0

-0.4516
-0.2610
-0.6422
[torch.FloatTensor of size 3x1]

1

-0.9455
-0.4676
-0.6638
[torch.FloatTensor of size 3x1]



In [83]:
class Sequence(nn.Module):
    def __init__(self):
        super(Sequence, self).__init__()
        #self.lstm1 = nn.LSTMCell(1, 51)
        #self.lstm2 = nn.LSTMCell(51, 51)
        #self.linear = nn.Linear(51, 1)
        
        self.lstm1 = nn.LSTMCell(3567, 51)
        self.lstm2 = nn.LSTMCell(51, 3)
        self.linear = nn.Linear(3, 3)

    def forward(self, input, future = 0):
        outputs = []
        #input = input.transpose(1,0)
        #print(input)
        #h_t = Variable(torch.zeros(input.size(0), 51).double(), requires_grad=False)
        #c_t = Variable(torch.zeros(input.size(0), 51).double(), requires_grad=False)
        #h_t2 = Variable(torch.zeros(input.size(0), 51).double(), requires_grad=False)
        #c_t2 = Variable(torch.zeros(input.size(0), 51).double(), requires_grad=False)
        h_t = Variable(torch.zeros(1, 51).double(), requires_grad=False)
        c_t = Variable(torch.zeros(1, 51).double(), requires_grad=False)
        h_t2 = Variable(torch.zeros(1, 3).double(), requires_grad=False)
        c_t2 = Variable(torch.zeros(1, 3).double(), requires_grad=False)
        for i, input_t in enumerate(input.chunk(input.size(1), dim=1)):
            #print(i)
            #print('input:', input_t.transpose(1,0).size())   
            h_t, c_t = self.lstm1(input_t.transpose(1,0), (h_t, c_t))
            #print('ht:', h_t.size())
            #print('ct:', c_t.size())  

            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
            #print('ht2:', h_t2)
        for i in range(future):# if we should predict the future
            h_t, c_t = self.lstm1(output, (h_t, c_t))
            h_t2, c_t2 = self.lstm2(h_t, (h_t2, c_t2))
            output = self.linear(h_t2)
            outputs += [output]
        outputs = torch.stack(outputs, 1).squeeze(2)
        return outputs


In [ ]:
if __name__ == '__main__':
    # set random seed to 0
    np.random.seed(0)
    torch.manual_seed(0)
    
    data0 = pd.read_csv('arrhythmia_data2.csv', header=None, low_memory=False)
    data = data0[1:3568]
    datat = data0.transpose()
    labels = datat[0]
    #print(range(len(data)))
    #print(len(datat))
    
    epocha = []
   
    seq = Sequence()
    seq.double()
    criterion = nn.MSELoss()
    # use LBFGS as optimizer since we can load the whole data to train
    optimizer = optim.SGD(seq.parameters(), lr=0.01)
    mepoch = 75
    input_dat = torch.DoubleTensor(data.values.astype(float))
    #print(input_dat)
    target_matrix = torch.zeros(3,77)
    trainsumerrora = []
    
    for batch in range(77):
        target_matrix[labels[batch], batch] = 1
    
    targetV = Variable(target_matrix.double())
    
    for epoch in range(0, mepoch):
        epocha.append(epoch)
        optimizer.zero_grad()
        out = seq(Variable(input_dat))
        #print('output', out)
        loss = criterion(out, targetV)
        print('loss:', loss.data.numpy()[0])
        loss.backward()
        trainsumerrora.append(loss[0].data.numpy()[0])
        optimizer.step()
        
    print(trainsumerrora)
    #print(epocha)
    
    #fig, ax = plt.subplots()
    #ax.plot(epocha, trainsumerrora, 'r', label = 'Training Error')
    #legend = ax.legend()
    plt.plot(epocha, trainsumerrora,'r')
    

loss: 0.560866317321
loss: 0.551781840862
loss: 0.543633016953
loss: 0.527722816898
loss: 0.522625940281
loss: 0.517625069479
loss: 0.512716198725
loss: 0.507897271989
loss: 0.503166293943
loss: 0.498521326953
loss: 0.493960488828
loss: 0.489481950798
loss: 0.485083935608
loss: 0.480764715702
loss: 0.476522611487
loss: 0.472355989668
loss: 0.468263261651
loss: 0.464242882008
loss: 0.460293347002
loss: 0.456413193174
loss: 0.452600995979
loss: 0.448855368478
loss: 0.44517496008
loss: 0.44155845533
loss: 0.438004572744
loss: 0.434512063685
loss: 0.431079711285
loss: 0.427706329397
loss: 0.424390761583
loss: 0.42113188012
loss: 0.417928585007
loss: 0.414779802888
loss: 0.41168448565
loss: 0.408641607496
loss: 0.405650151816
loss: 0.402708923117
loss: 0.399721366351
loss: 0.396398748536
loss: 0.393602709541
loss: 0.390853555235
loss: 0.388150420118
loss: 0.385492458295
loss: 0.382878842886
loss: 0.380308765472
loss: 0.377781435549
loss: 0.375296080007
loss: 0.372851942633


In [113]:

plt.plot(epocha, trainsumerrora,'r')
plt.show()

In [66]:
np.random.seed(2)

T = 20
L = 1000
N = 100

x = np.empty((N, L), 'int64')
x[:] = np.array(range(L)) + np.random.randint(-4 * T, 4 * T, N).reshape(N, 1)
data = np.sin(x / 1.0 / T).astype('float64')
torch.save(data, open('traindata.pt', 'wb'))

In [76]:
if __name__ == '__main__':
    # set random seed to 0
    np.random.seed(0)
    torch.manual_seed(0)
    # load data and make training set
    data = torch.load('traindata.pt')
    input = Variable(torch.from_numpy(data[3:, :-1]), requires_grad=False)
    target = Variable(torch.from_numpy(data[3:, 1:]), requires_grad=False)
    test_input = Variable(torch.from_numpy(data[:3, :-1]), requires_grad=False)
    test_target = Variable(torch.from_numpy(data[:3, 1:]), requires_grad=False)
    # build the model
    seq = Sequence()
    seq.double()
    criterion = nn.MSELoss()
    # use LBFGS as optimizer since we can load the whole data to train
    optimizer = optim.LBFGS(seq.parameters(), lr=0.8)
    #begin to train
    for i in range(15):
        print('STEP: ', i)
        def closure():
            optimizer.zero_grad()
            out = seq(input)
            loss = criterion(out, target)
            print('loss:', loss.data.numpy()[0])
            loss.backward()
            return loss
        optimizer.step(closure)
        # begin to predict
        future = 1000
        pred = seq(test_input, future = future)
        loss = criterion(pred[:, :-future], test_target)
        print('test loss:', loss.data.numpy()[0])
        y = pred.data.numpy()
        # draw the result
        plt.figure(figsize=(30,10))
        plt.title('Predict future values for time sequences\n(Dashlines are predicted values)', fontsize=30)
        plt.xlabel('x', fontsize=20)
        plt.ylabel('y', fontsize=20)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        def draw(yi, color):
            plt.plot(np.arange(input.size(1)), yi[:input.size(1)], color, linewidth = 2.0)
            plt.plot(np.arange(input.size(1), input.size(1) + future), yi[input.size(1):], color + ':', linewidth = 2.0)
        draw(y[0], 'r')
        draw(y[1], 'g')
        draw(y[2], 'b')
        plt.savefig('predict%d.pdf'%i)
        plt.close()

STEP:  0
Variable containing:
-0.9613 -0.9738 -0.9840  ...  -0.7353 -0.7683 -0.7993
-0.2474 -0.1987 -0.1494  ...  -0.6560 -0.6174 -0.5774
 0.9320  0.9490  0.9636  ...   0.6702  0.7065  0.7410
          ...             ⋱             ...          
-0.2474 -0.1987 -0.1494  ...  -0.6560 -0.6174 -0.5774
 0.9975  0.9998  0.9996  ...   0.8596  0.8841  0.9063
-0.9975 -0.9927 -0.9854  ...  -0.9231 -0.9412 -0.9569
[torch.DoubleTensor of size 97x999]

0
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
1
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
2
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
3
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
4
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
5
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
6
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
7
input: torch.S

190
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
191
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
192
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
193
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
194
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
195
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
196
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
197
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
198
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
199
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
200
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
201
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
202
input: torch.Size([1, 97])
ht: torch.Size([1, 51

ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
419
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
420
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
421
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
422
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
423
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
424
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
425
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
426
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
427
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
428
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
429
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
430
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
431


643
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
644
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
645
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
646
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
647
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
648
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
649
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
650
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
651
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
652
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
653
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
654
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
655
input: torch.Size([1, 97])
ht: torch.Size([1, 51

846
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
847
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
848
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
849
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
850
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
851
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
852
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
853
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
854
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
855
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
856
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
857
input: torch.Size([1, 97])
ht: torch.Size([1, 51])
ct: torch.Size([1, 51])
858
input: torch.Size([1, 97])
ht: torch.Size([1, 51

RuntimeError: input and target have different number of elements: input[1 x 999 x 3] has 2997 elements, while target[97 x 999] has 96903 elements at /opt/conda/conda-bld/pytorch_1513368888240/work/torch/lib/THNN/generic/MSECriterion.c:13